In [4]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import momepy as mm
import networkx as nx
from shapely import Point, LineString, reverse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from pyproj import Transformer
import datetime


import pandas as pd
import geopandas as gpd
from shapely.ops import unary_union
from pyproj import Transformer

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

c:\Users\ruben\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. consum de aigua agrupat per EDIFICI 

1.1 agrupem per EDIFICI

- fem mitjana data_lectura
 
- fem suma de CONSUM

 

In [5]:
# Load the data
consum = pd.read_csv('data/consum/consum.csv')
# Convert date columns to datetime format
consum['DATA LECTURA INI'] = pd.to_datetime(consum['DATA LECTURA INI'])
consum['DATALECTURA FI'] = pd.to_datetime(consum['DATALECTURA FI'])

# Compute the mean of the date columns for each building
#mean_dates = consum.groupby('EDIFICI')['DATA LECTURA INI', 'DATALECTURA FI'].mean()
mean_dates = consum.groupby('EDIFICI')[['DATA LECTURA INI', 'DATALECTURA FI']].mean()

# Rename columns to more meaningful names
mean_dates.columns = ['mean_lectura_ini', 'mean_lectura_fi']

# Group the consumption data by building and compute the total consumption
consum_edifici = consum.groupby('EDIFICI')['CONSUM'].sum().reset_index()

# Merge the consumption and date dataframes on the 'EDIFICI' column
consum_edifici = pd.merge(consum_edifici, mean_dates, on='EDIFICI')

# Print the resulting dataframe
print(consum_edifici.head())


   EDIFICI  CONSUM mean_lectura_ini mean_lectura_fi
0      2.0      32       2020-09-10      2021-01-15
1      3.0      43       2020-09-10      2021-01-15
2      5.0      28       2020-09-10      2021-01-15
3      6.0       0       2020-09-10      2021-01-15
4      9.0     135       2020-09-10      2021-01-15


C:\Users\ruben\AppData\Local\Temp\ipykernel_11684\3650283003.py:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  consum['DATA LECTURA INI'] = pd.to_datetime(consum['DATA LECTURA INI'])
C:\Users\ruben\AppData\Local\Temp\ipykernel_11684\3650283003.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  consum['DATALECTURA FI'] = pd.to_datetime(consum['DATALECTURA FI'])


1.2 associate to UMAT with commun index CODI_EDIFI

NOTE: that in edificis_umat one same CODI_EDIFI may have several ATRIBUT (strings) and geometry

 - in ATRIBUT  we put all the atributes togueter separated by , like Px+T,I+T
 
 - in geometry we calculate the superposition of all the POLYGON and print a single poligon

1.3 trobem centroide del poligon, extraiem longitud i latitud

1.4 calculem PERIODE de consum



In [6]:
# llegim els edificis del UMAT
edificis_umat = gpd.read_file('data/consum/edificis/edificis.shp')

# 1.2 Merge the data frames on the common index
merged = pd.merge(consum_edifici, edificis_umat, left_on='EDIFICI', right_on='CODI_EDIFI')

# Group the merged data by the common index, and combine the attributes into a single string
grouped = merged.groupby('CODI_EDIFI').agg({
    'CONSUM': 'sum',
    'ATRIBUT': lambda x: ','.join(x),
    'geometry': lambda x: unary_union(x),
    'SHAPE_AREA':'sum',
    'OBJECTID':lambda x: ','.join(str(i) for i in x),
    'mean_lectura_ini': lambda x: pd.to_datetime(x).mean(),
    'mean_lectura_fi': lambda x: pd.to_datetime(x).mean()
})

# Convert the grouped data back to a GeoDataFrame
result = gpd.GeoDataFrame(grouped, geometry='geometry')

# 1.3 Extract the centroid of each geometry
result['centroid'] = result.geometry.centroid

# Convert the coordinates of the centroid from EPSG:25831 to WGS84
transformer = Transformer.from_crs('EPSG:25831', 'EPSG:4326')
result['latitude'], result['longitude'] = transformer.transform(result.centroid.x, result.centroid.y)

# 1.4 Calculate the time differences in seconds and store them in a new column called 'PERIODE'
result['PERIODE'] = (result['mean_lectura_fi'] - result['mean_lectura_ini'].min()).dt.total_seconds()

# save the result
result.head()


,CONSUM,ATRIBUT,geometry,SHAPE_AREA,OBJECTID,mean_lectura_ini,mean_lectura_fi,centroid,latitude,longitude,PERIODE
CODI_EDIFI,,,,,,,,,,,
2.0,64,"I,III","POLYGON ((485674.577 4647942.149, 485674.378 4...",165.512811,"875,12574",2020-09-10,2021-01-15,POINT (485664.665 4647949.379),41.983416,2.826950,1.896826e+09
3.0,172,"-I+IV+T,-I+V,III,-I+IV","POLYGON ((485673.255 4647980.942, 485657.993 4...",233.360891,"20918,17785,42952,7170",2020-09-10,2021-01-15,POINT (485670.238 4647986.229),41.983748,2.827016,1.896826e+09
5.0,56,"I+T,III","POLYGON ((485685.177 4648005.528, 485684.191 4...",126.013930,"41315,35410",2020-09-10,2021-01-15,POINT (485678.831 4648004.349),41.983911,2.827119,1.896826e+09
6.0,0,III,"POLYGON ((485686.089 4648012.111, 485685.505 4...",64.673028,10889,2020-09-10,2021-01-15,POINT (485679.188 4648011.024),41.983971,2.827124,1.896826e+09
9.0,810,"IV,Px+III,IV,-I+IV,II+T,II","POLYGON ((485687.156 4648019.939, 485675.597 4...",197.435127,"2831,9123,13538,28932,32373,50878",2020-09-10,2021-01-15,POINT (485683.869 4648028.151),41.984126,2.827180,1.896826e+09


1.5 importem els manholes i trobem long lat

1.6 spatial join to find the closest manhole to each location

1.7 creem el dwf sumant els consums associats a cada node, afegim atributs

In [34]:
consum_geo=result

# Convert the latitude and longitude columns of consum_geo to a shapely Point object
geometry = [Point(long, lat) for lat, long in zip(consum_geo['latitude'], consum_geo['longitude'])]
locations = gpd.GeoDataFrame(consum_geo, geometry=geometry)

#1.5 importem els manholes
nodes_corregits = gpd.read_file("results/networks/nodes_corretgits.gpkg")
manholes = nodes_corregits[(nodes_corregits['sys_type']=='MANHOLE') & (nodes_corregits['state'] != 0)]   # Filter by MANHOLE sys_type

#transformem les coordenades dels manholes a long, lat
transformer = Transformer.from_crs("EPSG:25831", "WGS84")
for index, row in manholes.iterrows():
     #if row['state'] != 0:
         x = row['geometry'].x
         y = row['geometry'].y

         long, lat = transformer.transform(x, y)
         row['longitude'] = long
         row['latitude'] = lat
         row['x'] = x
         row['y'] = y
        
         row['geometry'] = (long, lat)

# 1.6 Perform a spatial join to find the closest manhole to each location
joined = gpd.sjoin_nearest(manholes, locations, how='left')


# Calculate the distance between each location and its closest manhole
#joined['distance'] = joined.apply(lambda x: distance((x['latitude'], x['longitude']), (x['geometry'].y, x['geometry'].x)).meters, axis=1)

#1.7 creem el dwf
manholes_dwf = joined.groupby('node_id').agg({
    'CONSUM': 'sum',
    'PERIODE': 'mean'
}).reset_index()

# Add the manhole code to the manholes_dwf DataFrame
manholes_dwf = manholes_dwf.merge(manholes[['node_id', 'code']], on='node_id', how='left')

# Add the columns pat1, value, dscencario, and new_id with NaN values

cm3=1
m3 = 1000000*cm3

manholes_dwf['pat1'] = 'hora'
manholes_dwf['value'] = 0.8 * manholes_dwf['CONSUM'] * m3 / manholes_dwf['PERIODE']
manholes_dwf['dwfscencario_id'] =2
manholes_dwf['node_id_new'] = float('nan')


manholes_dwf = manholes_dwf.rename(columns={'node_id': 'node_id_old'})
manholes_dwf.to_csv('data/consum/manholes_dwf.csv', index=False)

manholes_dwf.head()


C:\Users\ruben\AppData\Local\Temp\ipykernel_11684\4014815828.py:27: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: GEOGCS["Undefined geographic SRS",DATUM["unknown", ...
Right CRS: None

  joined = gpd.sjoin_nearest(manholes, locations, how='left')
c:\Users\ruben\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\array.py:344: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,node_id_old,CONSUM,PERIODE,code,pat1,value,dwfscencario_id,node_id_new
0,10,3480,1.897776e+09,PR0005291,hora,1.466980,2,NaN
1,10018,15639,1.899729e+09,PR0023125,hora,6.585783,2,NaN
2,1002,539,1.905984e+09,PR0024234,hora,0.226235,2,NaN
3,10020,15639,1.899729e+09,PR0023124,hora,6.585783,2,NaN
4,10022,112,1.899590e+09,PR0023123,hora,0.047168,2,NaN


we use the diccionary to fill the node_id with the new giswater index

In [37]:

manholes_dwf=pd.read_csv('data/consum/manholes_dwf.csv')
dic_node_id = pd.read_csv("data/networks/dic_node_id.csv")


# Merge the DataFrames on the 'code' column
manholes_dwf = pd.merge(manholes_dwf, dic_node_id[['code', 'node_id_new']], on='code', how='left').head()


manholes_dwf = manholes_dwf.rename(columns={'node_id_new_y': 'node_id'})

# drop unecessary columns
manholes_dwf = manholes_dwf.drop(columns=['node_id_new_x','node_id_old','CONSUM', 'PERIODE'])


# Save the manholes_dwf DataFrame to a CSV file
inp_dwf=manholes_dwf
inp_dwf.to_csv('data/consum/inp_dwf.csv', index=False)

inp_dwf.head()


,code,pat1,value,dwfscencario_id,node_id
0,PR0005291,hora,1.466980,2,13026
1,PR0023125,hora,6.585783,2,14499
2,PR0024234,hora,0.226235,2,13148
3,PR0023124,hora,6.585783,2,14500
4,PR0023123,hora,0.047168,2,14501


# 2. Consum de aigua geolocalitzat per adressa

1. agrupem els contadors amb igual ('NOM CARRER') i ('NUM')

 - sumem els consums i calculem temps de factura en segons

In [ ]:
# Load the csv file into a Pandas dataframe
consum = pd.read_csv('data/consum/consum.csv')

print(consum.columns)

#dwf = consum[['NOM CARRER', 'NUM','EDIFICI', 'CONSUM','DATA LECTURA INI','DATALECTURA FI']]

# Convert the 'DATA LECTURA INI' column to a datetime format
consum['DATA LECTURA INI'] = pd.to_datetime(consum['DATA LECTURA INI'],dayfirst=True)
consum['DATALECTURA FI'] = pd.to_datetime(consum['DATALECTURA FI'], dayfirst=True)

# Group the rows by 'NOM CARRER' and 'NUM', and calculate the sum of 'CONSUM' for each group
consum_grouped = consum.groupby(['NOM CARRER', 'NUM']).agg({'CONSUM': 'sum'})

# Calculate the mean datetime values for 'DATA LECTURA INI' and 'DATALECTURA FI' for each group
consum_grouped['DATA LECTURA INI'] = consum.groupby(['NOM CARRER', 'NUM'])['DATA LECTURA INI'].mean()
consum_grouped['DATALECTURA FI'] = consum.groupby(['NOM CARRER', 'NUM'])['DATALECTURA FI'].mean()

# Reset the index to make the group keys ('NOM CARRER' and 'NUM') columns again
consum_grouped = consum_grouped.reset_index()

# Calculate the time differences in seconds and store them in a new column called 'PERIODE'
consum_grouped['PERIODE'] = (consum_grouped['DATALECTURA FI'] - consum_grouped['DATA LECTURA INI'].min()).dt.total_seconds()
consum_grouped.to_csv('data/consum/consum_grouped.csv', index=False)
consum_grouped

determine their latitude and longitude useing the Geopy library.

- separem les entrades per grups de 200 perque el porgrama no reventi

- we add try and esceptions for lost NUM lost NOM CARRER

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
import time
import requests

#TODO: i keep seeing the warning in jupiter notebook enevn i try to stop it
# what shall i do? i can send u the warning if u ask
# Suppress the 'SettingWithCopyWarning' warning
import warnings
warnings.filterwarnings('ignore', category=Warning, module='pandas')

# Define the filename of your input CSV file
input_file = 'data/consum/consum_grouped.csv'

# Define the filename of your output CSV file
output_file = 'data/consum/consum_grouped_located.csv'

# Define the chunksize (number of rows to process at a time)
chunksize = 50

# compute the total number of chunks
total_chunks = pd.read_csv(input_file, nrows=1).shape[0]
print('Please be patient, we will geolocalize addresses: ')
print('Total ( ',total_chunks//chunksize+1,' ) chunks with len = (',chunksize,')' )


# Create a geolocator object
geolocator = Nominatim(user_agent='my-app',timeout=50)

# Define a function to get the latitude and longitude of an address
# we try diferent convinations to find missing adresses


def get_lat_long(row):
    address = f"carrer {row['NOM CARRER']}, {row['NUM']}, Gerona, Spain"
    location = geolocator.geocode(address)
    if location:
        return (location.latitude, location.longitude)
    else:
        return None
    
def get_lat_long_dif(row):
    # Split the street name into separate words
    words = row['NOM CARRER'].split()
    
    # Create a list of all possible combinations of the words, with one word removed in each combination
    combinations = []
    for i in range(len(words)):
        combination = ' '.join(words[:i] + words[i+1:])
        combinations.append(combination)
    
    # Try geocoding each combination
    for combination in combinations:
        address = f"carrer {combination}, {row['NUM']}, Gerona, Spain"
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
    
    # If none of the combinations work, return None
    return None
    
def get_lat_long_difSN(row):
    # Split the street name into separate words
    words = row['NOM CARRER'].split()
    
    # Create a list of all possible combinations of the words, with one word removed in each combination
    combinations = []
    for i in range(len(words)):
        combination = ' '.join(words[:i] + words[i+1:])
        combinations.append(combination)
    
    # Try geocoding each combination
    for combination in combinations:
        address = f"carrer {combination}, Gerona, Spain"
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
    
    # If none of the combinations work, return None
    return None

def get_lat_long_SN(row):
    # Remove NUM from the address
    address = f"carrer {row['NOM CARRER']}, Gerona, Spain"
    location = geolocator.geocode(address)
    if location:
        return (location.latitude, location.longitude)
    else:
        return None


# Define a function to process a chunk of the DataFrame and write the results to a CSV file
def process_chunk(chunk, i, max_retries=20):
    # Apply the function to the dataframe to get the latitude and longitude of each address
    for retry in range(max_retries):
        try:
            start_time = time.time()
            chunk['lat_long'] = chunk.apply(get_lat_long, axis=1)
            end_time = time.time()

            # Print the time taken to geocode the addresses in this chunk
            print(f"Time taken for the {i}/{total_chunks//chunksize+1} geocode with ( {len(chunk)} ) addresses: {end_time - start_time} seconds")

    
            # recompute the values with none lat_long without NUM
            if chunk['lat_long'].isnull().any():
            # Recompute the latitude and longitude values without NUM
                without_num = chunk[chunk['lat_long'].isnull()]
                print('!  full adresses not localized', len(without_num))


                if len(without_num) > 0:
                    without_num.loc[:, 'lat_long'] = without_num.apply(get_lat_long_dif, axis=1)
                    chunk.loc[without_num.index, 'lat_long'] = without_num['lat_long']
                    without_num = chunk[chunk['lat_long'].isnull()]
                    print('!!  dif adress not localized', len(without_num))

                if len(without_num) > 0:
                    without_num.loc[:, 'lat_long'] = without_num.apply(get_lat_long_SN, axis=1)
                    chunk.loc[without_num.index, 'lat_long'] = without_num['lat_long']
                    without_num = chunk[chunk['lat_long'].isnull()]
                    print('!!!  SN adress not localized', len(without_num))

                if len(without_num) > 0:
                    without_num.loc[:, 'lat_long'] = without_num.apply(get_lat_long_difSN, axis=1)
                    chunk.loc[without_num.index, 'lat_long'] = without_num['lat_long']
                    without_num = chunk[chunk['lat_long'].isnull()]
                    print('!!!!  dif+SN adress not localized', len(without_num))
                
                if len(without_num) == 0:
                    print('-> all adresses have been localized')

                if len(without_num) != 0:
                    print('ERROR  (',len(without_num),')  adress not localized')


            # Extract the latitude and longitude into separate columns
            chunk['latitude'] = chunk['lat_long'].apply(lambda x: x[0] if x else None)
            chunk['longitude'] = chunk['lat_long'].apply(lambda x: x[1] if x else None)
            # Save the chunk as a new .csv file
            if i == 1:
                chunk.to_csv(output_file, mode='w', index=False)
            else:
                chunk.to_csv(output_file, mode='a', index=False, header=False)

            
            # Exit the retry loop if the chunk was processed successfully
            break

        except Exception as e:
            print(f"An error occurred while processing chunk ( {i} ), retrying ({retry+1}/{max_retries})...")
            print(f"Error message: {e}")
    
    # Raise an exception if the maximum number of retries was reached without success
    else:
     #raise Exception(f"Failed to process chunk ( {i} ) after {max_retries} retries")
     #TODO is the following a good alternative?, i want to save none values in longitude, latitude, lat_long
     # in order to fill if unexistent(?) fields and continue for the next chunk 


        print(f"Failed to process chunk ( {i} ) after {max_retries} retries")
        # Extract the latitude and longitude into separate columns
        #chunk['lat_long'] = [(None,None)] * len(chunk)
        chunk['latitude'] = chunk['lat_long'].apply(lambda x: x[0] if x else None)
        chunk['longitude'] = chunk['lat_long'].apply(lambda x: x[1] if x else None)
        #TODO not sure if last 3 lines are necessary 
        # Save the chunk as a new .csv file
        if i == 1:
            chunk.to_csv(output_file, mode='w', index=False)
        else:
            chunk.to_csv(output_file, mode='a', index=False, header=False)
        
        #raise Exception(f"Failed to process chunk ( {i} ) after {max_retries} retries")

# Process the DataFrame in chunks of chunksize rows

#TODO if the values at 'latitude' of the output file (in case that it exisit) are all real fload jump to the next chunk 
i = 1
for chunk in pd.read_csv(input_file, chunksize=chunksize):
    process_chunk(chunk, i)
    i += 1



veiem les adresses conflitives

In [ ]:
import pandas as pd

# Read in the data
data = pd.read_csv('data/consum/consum_grouped_located.csv')

# Find all the NaN values in the 'latitude' column
nan_rows = data[data['latitude'].isna()]

print(len(nan_rows))
# Loop through each NaN row and display the corresponding street and number
#for _, row in nan_rows.iterrows():
#    print(f"Customize NaN value for: {row['NOM CARRER']}, {row['NUM']}")
    # try to geolocalize the longitude and latitude of thois street by using some python librtary


In [ ]:
import requests

def get_lat_long_API(address):
    url = 'https://nominatim.openstreetmap.org/search'
    #url = 'https://maps.googleapis.com/maps/api/geocode/json'
    #url = 'https://dev.virtualearth.net/REST/v1/Locations'


    params = {
        'q': address,
        'format': 'json'
    }
    response = requests.get(url, params=params)
    data = response.json()
    if len(data) > 0:
        lat = data[0]['lat']
        lon = data[0]['lon']
        return float(lat), float(lon)
    else:
        return None

#address = '1600 Amphitheatre Parkway, Mountain View, CA'
address = '7 , AVDA. RAMON FOLCH, Girona, Spain'
result = get_lat_long_API(address)
if result is not None:
    lat, lon = result
    print(f'The latitude and longitude of {address} are: {lat}, {lon}')
else:
    print(f'Could not find the latitude and longitude of {address}')




In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim

# Read in the data
data = pd.read_csv('data/consum/consum_grouped_located.csv')

# Find all the NaN values in the 'latitude' column
nan_rows = data[data['latitude'].isna()]

# Set up a geolocator object using Nominatim
geolocator = Nominatim(user_agent='my-app')

# Loop through each NaN row and display the corresponding street and number
for _, row in nan_rows.iterrows():
    # Get the address string for the street and number
    address = f"{row['NOM CARRER']}, Girona, Spain"
    # Use geopy to geolocalize the address
    
    try: 
        lat, lon = get_lat_long_API(address)
        
        if type(lat)!=float:  #TODO we may try another appy here instead
            location = geolocator.geocode(address)
            lat=location.latitude
            loc=location.longitude
    except:
        lat=None
        lon=None


    #TODO we may try another api here instead
    try: 
        location = geolocator.geocode(address)
        lat=location.latitude
        loc=location.longitude
    except:
        lat=None
        lon=None



    if type(lat)==float:
        print(lat,'   ',lon)
        data.loc[(data['NUM']==row['NUM']) & (data['NOM CARRER']==row['NOM CARRER']), ['latitude', 'longitude']] = [lat, lon]
        #TODO: if found, update the values in data['latitude'], data['longitude'] from my file .csv
    else:
        print(f"No location found for: {address}")

# Save the updated data to a new CSV file
data.to_csv('data/consum/consum_grouped_located_updated.csv', index=False)
    


associem el consum de cada edifici al manhole mes proper del nostre subgraph

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
# from geopy.distance import distance
# from geopandas.tools import nearest_neighbor

# Load the csv file into a Pandas dataframe
dwf0 =  pd.read_csv('data/consum/consum_grouped_located_updated.csv')
# Find all the not NaN values in the 'latitude' column
dwf = dwf0[dwf0['latitude'].notna()]

node_corregit = gpd.read_file("results/networks/nodes_corretgits.gpkg")
manholes = node_corregit[(node_corregit['sys_type']=='MANHOLE') & (node_corregit['state'] != 0)]   # Filter by MANHOLE sys_type

# Convert the latitude and longitude columns of dwf to a shapely Point object
geometry = [Point(long, lat) for lat, long in zip(dwf['latitude'], dwf['longitude'])]
locations = gpd.GeoDataFrame(dwf, geometry=geometry)

#transformem les coordenades
transformer = Transformer.from_crs("EPSG:25831", "WGS84")

#afegim els nodes
for index, row in manholes.iterrows():
     #if row['state'] != 0:
         x = row['geometry'].x
         y = row['geometry'].y

         long, lat = transformer.transform(x, y)
         row['longitude'] = long
         row['latitude'] = lat
         row['x'] = x
         row['y'] = y
        
         row['geometry'] = (long, lat)

# Perform a spatial join to find the closest manhole to each location
joined = gpd.sjoin_nearest(manholes, locations, how='left')


# Calculate the distance between each location and its closest manhole
#joined['distance'] = joined.apply(lambda x: distance((x['latitude'], x['longitude']), (x['geometry'].y, x['geometry'].x)).meters, axis=1)

# create a new df manholes_dwf where we add all the 'value' form my dwf['value'] with same manhole_id, 
# we should get a list of manholes with sum_value atribute with the sum of all the closest dwf
manholes_dwf = joined.groupby('node_id').agg({'CONSUM': 'sum'}).reset_index()

#to add the code
manholes_dwf = manholes_dwf.merge(manholes[['node_id', 'code']], on='node_id', how='left')


# Save the joined dataframe to a new csv file
manholes_dwf.to_csv('data/manholes_dwf.csv', index=False)